In [2]:
import numpy as np
import pandas as pd
import os
import csv
import ast
import seaborn as sns
from matplotlib import pyplot as plt
import wquantiles 

from tqdm import tqdm
from wquantiles import quantile
from openfisca_survey_manager.utils import asof

from openfisca_france_indirect_taxation import FranceIndirectTaxationTaxBenefitSystem
from openfisca_france_indirect_taxation.surveys import SurveyScenario
from openfisca_france_indirect_taxation.examples.utils_example import (
    dataframe_by_group,
    df_weighted_average_grouped,
    wavg)
from openfisca_france_indirect_taxation.build_survey_data.utils import weighted_sum
from openfisca_france_indirect_taxation.projects.TVA_Herve_IPP.new_calage_bdf_cn import new_get_inflators_by_year, new_get_cn_aggregates
from openfisca_france_indirect_taxation.projects.TVA_Herve_IPP.Utils import weighted_quantiles
from openfisca_france_indirect_taxation.utils import assets_directory, get_input_data_frame

In [104]:
data_tourisme_file_path = os.path.join(
    assets_directory,
    'legislation',
    'sect-tour-conso-int-conso.xlsx'
    )

In [105]:
postes_tourisme = ['Hébergements touristiques marchands',
 'Restaurants et cafés',
 'Transports par avion',
 'Transports par train',
 'Transports par autocar',
 'Transports fluviaux et maritimes',
 'Location de véhicules de tourisme',
 'Remontées mécaniques',
 'Musées, spectacles et autres activités culturelles',
 'Activités sportives et de loisirs',
'Location d\'articles de sport et loisirs',
'Services des voyagistes et agences de voyages',
'Carburants et péages',	
'Aliments et boissons',	
'Biens de consommation durables spécifiques',	
'Autres biens de consommation et autres services',
'Dépenses touristiques intérieures (C = A + B)'
 ]

In [211]:
data_tourisme = pd.read_excel(data_tourisme_file_path, header = [3,4])
data_tourisme.set_index([('Poste de dépenses','Unnamed: 0_level_1')], inplace= True)
target_label = "Consommation des non-résidents (consommation récepteur)"
data_tourisme = data_tourisme.loc[:, data_tourisme.columns.get_level_values(1) == target_label]
data_tourisme.columns = data_tourisme.columns.get_level_values(0)
data_tourisme.reset_index(inplace = True)
data_tourisme.rename({('Poste de dépenses', 'Unnamed: 0_level_1') : 'Poste de dépenses'}, axis = 1, inplace = True)
data_tourisme['Poste de dépenses'] = data_tourisme['Poste de dépenses'].str.strip() 

data_tourisme = data_tourisme.loc[data_tourisme['Poste de dépenses'].isin(postes_tourisme),]

In [212]:
other_data_tourisme = pd.read_excel(os.path.join(assets_directory,'legislation','Consommation_touristique_2010_2018.xlsx'))
other_data_tourisme['Postes de dépenses'] = other_data_tourisme['Postes de dépenses'].str.strip()

In [213]:
carburants_2018 = float(other_data_tourisme.loc[other_data_tourisme['Postes de dépenses'] == 'Carburants','2018'])
peages_2018 = float(other_data_tourisme.loc[other_data_tourisme['Postes de dépenses'] == 'Péages','2018'])
part_carburant = carburants_2018 / (carburants_2018 + peages_2018)
part_peages = 1 - part_carburant

data_tourisme.loc[data_tourisme.index.max()+1,'Poste de dépenses'] = 'Carburants'
data_tourisme.loc[data_tourisme['Poste de dépenses'] == 'Carburants',[2019,2020,2021,2022]] = (data_tourisme.loc[data_tourisme['Poste de dépenses']== 'Carburants et péages',[2019,2020,2021,2022]].
                                                                                               apply(lambda x : part_carburant*x, axis = 0)).values[0]
data_tourisme.loc[data_tourisme.index.max()+1,'Poste de dépenses'] = 'Peages'
data_tourisme.loc[data_tourisme['Poste de dépenses'] == 'Peages',[2019,2020,2021,2022]] = (data_tourisme.loc[data_tourisme['Poste de dépenses']== 'Carburants et péages',[2019,2020,2021,2022]].
                                                                                               apply(lambda x : (1-part_carburant)*x, axis = 0)).values[0]


In [214]:
autres_biens_conso_2018 = float(other_data_tourisme.loc[other_data_tourisme['Postes de dépenses'] == 'Autres biens de consommation (6)','2018'])
autres_services_2018 = float(other_data_tourisme.loc[other_data_tourisme['Postes de dépenses'] == 'Autres services (7)','2018'])
taxis_2018 = float(other_data_tourisme.loc[other_data_tourisme['Postes de dépenses'] == 'Taxis et autres services de transports urbains','2018'])
autres_biens_services_2018 = autres_biens_conso_2018 + autres_services_2018 + taxis_2018 

part_biens_conso = autres_biens_conso_2018 / autres_biens_services_2018
part_services = autres_services_2018 / autres_biens_services_2018
part_taxis = taxis_2018 / autres_biens_services_2018

data_tourisme.loc[data_tourisme.index.max()+1,'Poste de dépenses'] = 'Autres biens de consommation'
data_tourisme.loc[data_tourisme['Poste de dépenses'] == 'Autres biens de consommation',[2019,2020,2021,2022]] = (data_tourisme.loc[data_tourisme['Poste de dépenses'] == 
                                                                                             'Autres biens de consommation et autres services',[2019,2020,2021,2022]].
                                                                                            apply(lambda x : part_biens_conso*x, axis = 0)).values[0]

data_tourisme.loc[data_tourisme.index.max()+1,'Poste de dépenses'] = 'Autres services'
data_tourisme.loc[data_tourisme['Poste de dépenses'] == 'Autres services',[2019,2020,2021,2022]] = (data_tourisme.loc[data_tourisme['Poste de dépenses'] == 
                                                                                'Autres biens de consommation et autres services',[2019,2020,2021,2022]].
                                                                                apply(lambda x : part_services*x, axis = 0)).values[0]

data_tourisme.loc[data_tourisme.index.max()+1,'Poste de dépenses'] = 'Taxis et autres services de transports urbains'
data_tourisme.loc[data_tourisme['Poste de dépenses'] == 'Taxis et autres services de transports urbains',[2019,2020,2021,2022]] = (data_tourisme.loc[data_tourisme['Poste de dépenses'] == 
                                                                                                            'Autres biens de consommation et autres services',[2019,2020,2021,2022]].
                                                                                                            apply(lambda x : part_taxis*x, axis = 0)).values[0]

In [215]:
data_tourisme.set_index('Poste de dépenses', inplace = True)
data_tourisme.drop(index = ['Carburants et péages', 'Autres biens de consommation et autres services','Dépenses touristiques intérieures (C = A + B)'], axis = 1, inplace = True)

In [218]:
percentage_df = data_tourisme.div(data_tourisme.sum(axis = 0))
percentage_df_2022 = percentage_df[[2022]]
percentage_df_2022['Solde territorial (2022)'] = percentage_df_2022[2022] * solde_territorial

C:\Users\veve1\AppData\Local\Temp/ipykernel_26576/1316742971.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  percentage_df_2022['Solde territorial (2022)'] = percentage_df_2022[2022] * solde_territorial


In [208]:
solde_territorial

-10913.0

In [219]:
percentage_df_2022.sort_values(by = 2022, ascending= False)

,2022,Solde territorial (2022)
Poste de dépenses,,
Hébergements touristiques marchands,0.370892,-4047.544601
Restaurants et cafés,0.137715,-1502.885759
Transports par avion,0.095462,-1041.773083
Aliments et boissons,0.087637,-956.381847
Carburants,0.052817,-576.390845
Activités sportives et de loisirs,0.050078,-546.503912
Location de véhicules de tourisme,0.032864,-358.643192
Transports par autocar,0.032864,-358.643192
Biens de consommation durables spécifiques,0.029734,-324.486698


In [ ]:
dico_postes_tourisme = {
# 'Aliments et boissons' : 'poste_01' , 'poste_02',
# 'Biens de consommation durables spécifiques' : 'poste_03_1_1', 'poste_03_1_2', 'poste_03_2_1',
 'Carburants' : 'poste_07_2_2', 
 'Peages' : 'poste_07_2_4',
 'Location de véhicules de tourisme' : 'poste_07_2_4',
 'Transports par train' : 'poste_07_3_1',
 'Transports par autocar' : 'poste_07_3_2',
 'Transports par avion' : 'poste_07_3_3',
 'Transports fluviaux et maritimes': 'poste_07_3_4',
 'Location d\'articles de sport et loisirs' : 'poste_09_4_2_2',
 'Remontées mécaniques' : 'poste_09_4_2_4',
 'Activités sportives et de loisirs' : 'poste_09_4_2_4',
 'Musées, spectacles et autres activités culturelles' : 'poste_09_6_1', 
 'Services des voyagistes et agences de voyages' : 'poste_09_8',
 'Restaurants et cafés' : 'poste_11_1_1',
 'Hébergements touristiques marchands' : 'poste_11_2',
 'Autres services': 'poste_13_9',
 'Taxis et autres services de transports urbains': 'poste_13_9',			
# 'Autres biens de consommation': , 
# ,
}

In [6]:
target_year = 2017

In [ ]:
parametres_fiscalite_file_path = os.path.join(
            assets_directory,
            'legislation',
            'conso_eff_fonction_2023.xls'
            )

masses_cn_data_frame = pd.read_excel(parametres_fiscalite_file_path, sheet_name = "MEURcour", header = 4)
masses_cn_data_frame.rename(columns={'Unnamed: 0' : 'Code' , 'Unnamed: 1' : 'Label'}, inplace = True)
masses_cn_data_frame = masses_cn_data_frame.loc[:, ['Code', '{}'.format(target_year)]].copy()
masses_cn_data_frame.replace(to_replace = ajust_postes_cn, inplace= True)
masses_cn_data_frame.loc[:,'Code'] = masses_cn_data_frame.loc[:,'Code'].str.replace(r'^CP','',regex=True)
masses_cn_data_frame.loc[:,'Code'] = masses_cn_data_frame.loc[:,'Code'].str.strip()

masses_cn_data_frame.dropna(inplace = True)
masses_cn_data_frame.loc[:,'Code'] = masses_cn_data_frame['Code'].astype(str).apply(lambda x: f"poste_{x}")
masses_cn_data_frame.loc[:,'Code'] = masses_cn_data_frame['Code'].astype(str).apply(lambda x: format_poste(x))

# On garde les agrégats à un niveau supérieur pour correspondre à Bdf
masses_cn_data_frame = masses_cn_data_frame[~masses_cn_data_frame['Code'].isin(['poste_04_2_1', 'poste_04_2_2',
                                                                                'poste_05_1_1','poste_05_1_2','poste_05_2_1', 'poste_05_2_2',
                                                                                'poste_06_1','poste_06_2','poste_06_3','poste_06_4'])]

# On regroupe certains postes de consommation sous la même étiquette
masses_cn_data_frame = sum_and_remove(data_frame = masses_cn_data_frame,
                                        col = 'Code',
                                        rows_to_sum = ['poste_08_1_1', 'poste_08_1_2'],
                                        new_row = 'poste_08_1' )

masses_cn_data_frame = sum_and_remove(data_frame = masses_cn_data_frame,
                                        col = 'Code',
                                        rows_to_sum = ['poste_09_4_2_1', 'poste_09_4_2_2', 'poste_09_4_2_3', 'poste_09_4_2_4'],
                                        new_row = 'poste_09_4_2' )

masses_cn_data_frame = sum_and_remove(data_frame = masses_cn_data_frame,
                                        col = 'Code',
                                        rows_to_sum = ['poste_09_6_3_1', 'poste_09_6_3_2'],
                                        new_row = 'poste_09_6_3' )

masses_cn_data_frame = sum_and_remove(data_frame = masses_cn_data_frame,
                                        col = 'Code',
                                        rows_to_sum = ['poste_10_2_1', 'poste_10_2_2'],
                                        new_row = 'poste_10_2' )
# On ajoute les lignes "Autres dépenses de ..." pour chaque poste agrégé et on les mets à 0
liste_postes_nuls = ['poste_01_3_1', 'poste_02_5_1', 'poste_03_3_1', 'poste_04_6_1' , 'poste_05_7_1', 'poste_07_1_4','poste_07_5_1',
                        'poste_08_4_1', 'poste_09_9_1', 'poste_10_6_1', 'poste_11_1_3', 'poste_12_3_3_1','poste_12_8_1', 'poste_12_9_1']
for poste in liste_postes_nuls :
        masses_cn_data_frame = sum_and_remove(masses_cn_data_frame, 'Code', [], poste)

# Correction dépenses de santé 
part_menages = get_reste_a_charge_sante_cn(target_year)
masses_cn_data_frame.loc[masses_cn_data_frame['Code'] == 'poste_06','{}'.format(target_year)] = part_menages * masses_cn_data_frame.loc[masses_cn_data_frame['Code'] == 'poste_06','{}'.format(target_year)]
liste_postes_cn = remove_prefixes(masses_cn_data_frame['Code'].tolist())

liste_13postes = ["poste_0{}".format(i) for i in range(1, 10)] + ["poste_10", "poste_11", "poste_12", "poste_13"]
liste_postes_cn = [element for element in liste_postes_cn if element[:8] in liste_13postes]
masses_cn_postes_data_frame = masses_cn_data_frame.loc[masses_cn_data_frame['Code'].isin(liste_postes_cn)]
masses_cn_postes_data_frame.set_index('Code', inplace = True)

solde_territorial = float(masses_cn_data_frame.loc[masses_cn_data_frame['Code'] == 'poste_16','{}'.format(target_year)])

In [42]:
masses_cn_postes_data_frame = masses_cn_data_frame.loc[masses_cn_data_frame['Code'].isin(liste_postes_cn)]
masses_cn_postes_data_frame.set_index('Code', inplace = True)

In [199]:
from openfisca_france_indirect_taxation.projects.TVA_Herve_IPP.new_calage_bdf_cn import new_get_inflators_bdf_to_cn

In [200]:
new_get_inflators_bdf_to_cn(2017)

c:\Users\veve1\miniconda3\envs\openfiscaenv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
c:\users\veve1\onedrive\documents\ensae 3a\memoire mie\openfisca-france-indirect-taxation\openfisca_france_indirect_taxation\projects\TVA_Herve_IPP\new_calage_bdf_cn.py:182: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_bdf_postes_cn[poste] = 0


{'poste_01_1_1': 1.0288551574737594,
 'poste_01_1_2': 1.1102187472242049,
 'poste_01_1_3': 0.8727957960706688,
 'poste_01_1_4': 1.1286142021082546,
 'poste_01_1_5': 1.2236731357224044,
 'poste_01_1_6': 1.0329001909792201,
 'poste_01_1_7': 1.2007684207369826,
 'poste_01_1_8': 1.5008816768838418,
 'poste_01_1_9': 3.488342685296373,
 'poste_01_2_1': 0.9699320608559848,
 'poste_01_2_2': 1.2735049139537253,
 'poste_01_2_3': 0.9170285005986617,
 'poste_01_2_5': 1.274563224757769,
 'poste_01_2_6': 1.641844644858161,
 'poste_01_2_9': 8.058526866017846,
 'poste_02_1_1': 1.9019808982449435,
 'poste_02_1_2': 2.2521652776790826,
 'poste_02_1_3': 2.1193103847911035,
 'poste_02_1_9': 0.1580365381603684,
 'poste_02_3': 1.7635919583735522,
 'poste_03_1_1': 1.0327877067847278,
 'poste_03_1_2': 1.3568074584587102,
 'poste_03_1_3': 3.2910993124722863,
 'poste_03_1_4': 0.8911875760453848,
 'poste_03_2_1': 0.8886778953372103,
 'poste_03_2_2': 1.7516907728529518,
 'poste_04_1_1': 1.028882815273506,
 'poste_